## Extract ACF and Train a SVM Classifier
Successfully tested on 2018-11-23

In [22]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt
% matplotlib inline
import random
import os, sys


from sklearn import svm, datasets, externals 

# Set to your datasets folder, and including the svm model
path = 'E:\DataSets\INRIAPerson'
retval = os.getcwd()
print ('****Please set to your datasets folder, and including the svm model')
print ("Original Work Dir:%s" % retval)
os.chdir(path)
retval = os.getcwd()
print("Change to Current Dir:%s" % retval)

****Please set to your datasets folder, and including the svm model
Original Work Dir:E:\DataSets\INRIAPerson
Change to Current Dir:E:\DataSets\INRIAPerson


### Read in Images for Training

In [23]:
# 单独写了一个读取96x160H96正样本的函数，取中间的80x160的部分
def load_pos_images(dirname, amout = 9999):
    img_list = []
    file = open(dirname)
    img_name = file.readline()
    while img_name != '':  # 文件尾        
        img=cv2.imread(img_name.strip('\n'))  #注意出去换行符
        img_crop=img[:,8:96-8,:]
        img_list.append(img_crop)
        img_name = file.readline()
        
        amout -= 1
        if amout <= 0: # 控制读取图片的数量
            break
    return img_list

def load_neg_images(dirname, amout = 9999):
    img_list = []
    file = open(dirname)
    img_name = file.readline()
    while img_name != '':  # 文件尾
        img_name = dirname.rsplit(r'/', 1)[0] + r'/' + img_name.split('/', 1)[1].strip('\n')
        #print(img_name)
        img_list.append(cv2.imread(img_name))
        img_name = file.readline()
        amout -= 1
        if amout <= 0: # 控制读取图片的数量
            break
    return img_list

# 从每一张没有人的原始图片中随机裁出1张64*128的图片作为负样本
def sample_neg(full_neg_lst, neg_list, size):
    random.seed(1)
    width, height = size[1], size[0]
    for i in range(len(full_neg_lst)):
        for j in range(10):
            y = int(random.random() * (len(full_neg_lst[i]) - height))
            x = int(random.random() * (len(full_neg_lst[i][0]) - width))
            neg_list.append(full_neg_lst[i][y:y + height, x:x + width])
    return neg_list


In [24]:
def cell_gradient(cell_magnitude, cell_angle,bin_size,angle_unit):
    orientation_centers = [0] * bin_size
    
    for k in range(cell_magnitude.shape[0]):
        for l in range(cell_magnitude.shape[1]):
            #在每个cell中，根据角度信息，向对应的Bin累加mag
            gradient_strength = cell_magnitude[k][l]
            gradient_angle = cell_angle[k][l]
            # 对6取余
            min_angle = int(gradient_angle / angle_unit)% 6
            max_angle = (min_angle + 1) % bin_size
            mod = gradient_angle % angle_unit
            # 根据权重，一部分累计角度所在的Bin,和下一个Bin
            orientation_centers[min_angle] += (gradient_strength * (1 - (mod / angle_unit)))
            orientation_centers[max_angle] += (gradient_strength * (mod / angle_unit))
    return orientation_centers


def ComptChannels(Orig_Img, WinSize = (80,160)):
    '''
    % 获得原图像的相应十幅特征图
    % 输入：原图像，opencv读入BGR, h*w*3 = 160*80*3
    % 输出：特征图像：m*n*10
    % 3*LUV + gradMag + 6*gradHist
    % color:LUV
    % gradMag:Gray
    % gradHist:Gray
    '''
    
    Crop_img =cv2.resize(Orig_Img,WinSize) # 注意cv2.resize是w*h，反的

    h = Crop_img.shape[0]
    w = Crop_img.shape[1]
    chnl = 10
    
    # Create Feature Channels
    FeatureChannels = np.empty((h, w, chnl))
    
    # 原始RGB图像
    # RGB_Img = cv2.cvtColor(Orig_Img,cv2.COLOR_BGR2RGB)
    # RGB_Img = cv2.normalize(Orig_Img.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)

    # LUV通道
    LUV_Img = cv2.cvtColor(Crop_img,cv2.COLOR_BGR2LUV)
    FeatureChannels[:,:,0:3] = LUV_Img[:,:,:]

    # 梯度特征
    # Gradient Magnitude 梯度幅度特征
    Gray_Img = cv2.cvtColor(Crop_img,cv2.COLOR_BGR2GRAY)
    gradient_values_x = cv2.Sobel(Gray_Img, cv2.CV_64F, 1, 0, ksize=5)
    gradient_values_y = cv2.Sobel(Gray_Img, cv2.CV_64F, 0, 1, ksize=5)
    #计算角度和幅值
    gradient_magnitude, gradient_angle = cv2.cartToPolar(gradient_values_x, gradient_values_y, angleInDegrees=True)

    FeatureChannels[:,:,3] = gradient_magnitude[:,:]

    # HoG特征 
    # Gradient Histograms 梯度直方图特征
    # 划分为4x4的cell，每个cell里求Hist of Gradient 梯度直方图
    height, width = Gray_Img.shape
    cell_size = np.int(4)

    hcell = int(np.ceil(height / cell_size))
    wcell = int(np.ceil(width / cell_size))

    bin_size = np.int(6)
    angle_unit = int(360/bin_size)
    gradient_magnitude = abs(gradient_magnitude)


    # 一个cell由4x4的像素组成
    # 用于存储每个Cell的Gradient Histo
    cell_gradient_vector = np.zeros((hcell, wcell, bin_size))

    #遍历每个20x20的每个cell，统计Gradient Histo
    for i in range(cell_gradient_vector.shape[0]):
        for j in range(cell_gradient_vector.shape[1]):

            # Index获取此cell的Gradient幅值和角度
            cell_magnitude = gradient_magnitude[i * cell_size:(i + 1) * cell_size,j * cell_size:(j + 1) * cell_size]
            cell_angle = gradient_angle[i * cell_size:(i + 1) * cell_size,j * cell_size:(j + 1) * cell_size]
            #print(cell_angle.max())

            # 计算此Cell的Gradient Histo
            cell_gradient_vector[i][j] = cell_gradient(cell_magnitude, cell_angle, bin_size,angle_unit)

            # 分配6个Bin至Feature Channel的4-9的6个通道
            for n in range (bin_size):
                cell_hist_copy = np.ones((cell_size,cell_size)) * cell_gradient_vector[i,j,n]
                #print(cell_hist_copy)
                FeatureChannels[i * cell_size:(i + 1) * cell_size,j * cell_size:(j + 1) * cell_size,n+4] = cell_hist_copy

    NormFeatureChannels = np.empty(FeatureChannels.shape)
    cv2.normalize(FeatureChannels,  NormFeatureChannels, 0, 1, cv2.NORM_MINMAX)
    
    SubChannels = cv2.resize(NormFeatureChannels,(20,40),interpolation=cv2.INTER_CUBIC)
    AcfVec = SubChannels.reshape(-1)
    return AcfVec

def computeACFs(img_lst, acf_lst):
    for i in range(len(img_lst)):
        NormFeatureChannels = ComptChannels(img_lst[i])
        #print(NormFeatureChannels.shape)
        #print(NormFeatureChannels[4:10,4:10,6])
        acf_lst.append(NormFeatureChannels)
    return acf_lst

In [25]:
# 主程序
# 从list读取正负训练样本
pos_list = []
neg_list = []
acf_lst = []
labels = []

# 正样本是160x96的，共2416张，从中件选取160x80的ROI
pos_list = load_pos_images(r'./96X160H96/Train/pos_crop.lst')

# 负样本尺寸是不固定的，共1218张，从中随机抽取10张，160x80的区域
full_neg_lst = load_neg_images(r'./Train/neg.lst')
sample_neg(full_neg_lst, neg_list, [160, 80])
print('PosList:',len(pos_list),'PosSampleSize:',pos_list[1].shape)
print('NegList:',len(neg_list),'NegSampleSize:',neg_list[1].shape)

PosList: 2416 PosSampleSize: (160, 80, 3)
NegList: 12180 NegSampleSize: (160, 80, 3)


In [26]:
# 计算ACF特征通道，这一步大概需要15分钟
computeACFs(pos_list,acf_lst)
[labels.append(+1) for _ in range(len(pos_list))]

computeACFs(neg_list,acf_lst)
[labels.append(-1) for _ in range(len(neg_list))]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [27]:
print('Acf List:',len(acf_lst),np.array(acf_lst).shape)
print('Each AcF',acf_lst[12].shape)
print('Label List:',len(labels),np.array(labels).shape)
#print(labels)

Acf List: 14596 (14596, 8000)
Each AcF (8000,)
Label List: 14596 (14596,)


In [28]:
### Train SVM
x_train = np.array(acf_lst)
y_label = np.array(labels)
print(x_train.shape)
print(y_label.shape)

# model可以继续调节
model = svm.SVC(kernel='linear',C=30,decision_function_shape='ovo')
print(model)

(14596, 8000)
(14596,)
SVC(C=30, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [29]:
# 训练SVM模型，大概5分钟
model.fit(x_train, y_label)
print(model.support_vectors_.shape)
externals.joblib.dump(model,'ACF_trained_model.m')

(1714, 8000)


['ACF_trained_model.m']

### Test

In [30]:
# 测试一张train的图片
print(acf_lst[2].shape)
print('Test One Positive Traning Image, Predict:',model.predict(acf_lst[2].reshape(1, -1)))

print('Test One Negative Traning Image, Predict:',model.predict(acf_lst[-2].reshape(1,-1)))


(8000,)
Test One Positive Traning Image, Predict: [1]
Test One Negative Traning Image, Predict: [-1]


In [31]:
print ('****** Start to load SVM models *******')
model = externals.joblib.load('ACF_trained_model_v1.0.m')
print(model)

****** Start to load SVM models *******
SVC(C=30, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [32]:
# 读取一张

# 读取正测试样本的函数,大小统一为134x70
test_list = []

def load_test_images(dirname, amout = 9999):
    img_list = []
    file = open(dirname)
    img_name = file.readline()
    while img_name != '':  # 文件尾     
        img=cv2.imread(img_name.strip('\n'))  #注意出去换行符
        img_list.append(img)
        img_name = file.readline()
        
        amout -= 1
        if amout <= 0: # 控制读取图片的数量
            break
    return img_list

test_list = load_test_images(r'.\Test\test_img_list_crop.Lst')
print('TestList:',len(test_list),'Test Sample 0:',test_list[0].dtype,test_list[0].shape)
print('TestList:',len(test_list),'Test Sample 8:',test_list[8].dtype,test_list[8].shape)

TestList: 565 Test Sample 0: uint8 (134, 70, 3)
TestList: 565 Test Sample 8: uint8 (134, 70, 3)


In [33]:
test_acf_lst=[]
computeACFs(test_list,test_acf_lst)

print('test_acf_list:',len(test_acf_lst),np.array(test_acf_lst).shape)

test_acf_list: 565 (565, 8000)


In [15]:
# 测试一张test的图片
print(test_acf_lst[2].shape)
print('Test One Positive Test Image, Predict:',model.predict(test_acf_lst[2].reshape(1, -1)))

(8000,)
Test One Positive Test Image, Predict: [-1]


In [34]:
# 测试整个test的图片集
prediction = []
CorrentCnt = np.int(0)
IncorrectCnt = np.int(0) 
IncorrectLabel=[]
for i in range(len(test_acf_lst)):
    prediction.append(model.predict(np.array(test_acf_lst[i]).reshape(1,-1)))

for i in range(len(prediction)):
    if prediction[i]==1:
        CorrentCnt = CorrentCnt + 1
    else:
        IncorrectCnt = IncorrectCnt + 1
        IncorrectLabel.append(i)
    #print(prediction[i])
print(CorrentCnt,IncorrectCnt,len(prediction),'Accuracy:',CorrentCnt/len(prediction))
print('Incorrect Label:',IncorrectLabel)

331 234 565 Accuracy: 0.5858407079646017
Incorrect Label: [2, 3, 10, 11, 14, 17, 25, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 50, 51, 52, 56, 57, 60, 61, 66, 67, 70, 72, 73, 74, 75, 80, 81, 83, 84, 85, 86, 87, 89, 90, 91, 93, 94, 95, 97, 98, 99, 100, 101, 102, 103, 105, 106, 107, 108, 110, 111, 112, 113, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 146, 156, 157, 160, 161, 162, 164, 165, 166, 167, 168, 169, 172, 174, 180, 183, 184, 189, 190, 191, 192, 193, 194, 199, 200, 201, 202, 205, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 220, 221, 222, 224, 225, 226, 227, 228, 229, 230, 237, 238, 239, 240, 241, 242, 245, 263, 267, 269, 270, 272, 275, 277, 278, 279, 280, 287, 288, 289, 290, 291, 292, 306, 307, 308, 309, 310, 319, 320, 321, 322, 335, 336, 337, 338, 341, 342, 343, 347, 348, 352, 355, 356, 377, 379, 380, 383, 384, 403, 404, 406, 410, 417, 423, 424, 429, 430, 437, 438, 439, 441, 442, 450, 453, 454, 463, 464, 465, 467, 46